In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 11 11:00:05 2023

@author: jordansiegel
"""
import pandas as pd
import os
import numpy as np
from pathlib import Path

path = Path(r"%s"%(os.getcwd()))
# read in participant list
current_dir = os.getcwd()









In [9]:
print(path)
print(current_dir)
print(os.listdir(path.parent))

/Users/jordansiegel/Documents/GitHub/WTP_Rejection_Choice/scoring
/Users/jordansiegel/Documents/GitHub/WTP_Rejection_Choice/scoring
['Archive_CSV', 'dataframe.js', 'Raw_Participant_Images', 'index.html', '.DS_Store', 'WTP_SM_Choice_legacy.js', 't_test_analyses.ipynb', 'Images', 'participantlist.xlsx', 'WTP-PostTask_December 3, 2024_18.14.csv', 'OLD_WTP_SM_Choice.psyexp', 'WTP_SM_Choice.py', 'WTP_SM_Choice_legacy.psyexp', 'shortformdata.csv', 'WTP_SM_Choice.psyexp', 'Rej-WTP_Trials.py', 'GenerateParticipantFolders.py', 'WTP_Pretask_December 3, 2024_18.11.csv', '.gitignore', 'WTP_trials_template.xlsx', '.github', 'shortformdata.py', 'scoring', 'WTP_SM_Choice_legacy-legacy-browsers.js', '.ipynb_checkpoints', '.git', 'Participant_Images', 'spreadsheet_template.csv', 'WTP_GenerateParticipantFolders.py', 'data', 'newshortformdata.py', 'WTP_SM_Choice_lastrun.py', 'analysis_scoring_script_examples']


In [10]:
participants = pd.read_excel('%s/participantlist.xlsx'%(path.parent))
participants = participants.loc[
    participants['PhotosUploaded? (y/n)'] == 'y'].reset_index()
participants = pd.DataFrame(data=participants['PROLIFIC_ID'])

posttask_survey = '%s/' %(str(path.parent)) + [posttask for posttask in os.listdir(path.parent) if posttask.startswith('WTP-PostTask')][0]
#read in raw qualtrics data
alldata = pd.read_csv(posttask_survey)
alldata = alldata.iloc[2:]
alldata = alldata.sort_values(by=['PROLIFIC_ID'])
alldata.pop("Attnchk")  # remove attention checks
alldata = alldata.reset_index()


In [13]:
#columns list
RR_cols = [col for col in alldata.columns if 'RR_' in col]
ProlificID_cols = [col for col in alldata.columns if 'Prolific_ID' in col]

RR = alldata.filter(regex='RR_|Prolific_ID')

#%%
RR_clean = pd.DataFrame()


for i in range(0, len(RR)):
    if RR.loc[i,'Prolific_ID'] in participants['PROLIFIC_ID'].values:
       RR_clean[i] = RR.loc[i]    

finaldata = pd.DataFrame()


RR_clean = RR_clean.transpose()
RR_clean = RR_clean.reset_index()
finaldata['Prolific_ID'] = RR_clean['Prolific_ID']
RR_clean = RR_clean.drop(['index'], axis = 1)
RR_clean = RR_clean.drop(['Prolific_ID'], axis = 1)
RR_clean = RR_clean.replace(np.nan, 0)

#%%
RR_score = pd.DataFrame(columns = RR_clean.columns, index = RR_clean.index)


#%%
for k in range(0,len(RR_clean)):
    for i in range (0,len(RR_clean.columns)):
        RR_score[RR_clean.columns[i]][k] = RR_clean.loc[k,RR_clean.columns[i]]
        
RR_score= RR_score.astype(int)
RR_score["RR_totalscore"] = RR_score.sum(axis=1)
#those with no data needs a total score column too
# if len(no_data) > 1:
#     RR_score.loc[len(RR_score)+1] = '-'
#     participants.loc(len(RR_score)+1) = no_data['PROLIFIC_ID'] #!!!!!!!
# RR_score.insert(0, 'PROLIFIC_ID', participants)
#%%








#%%
rr = pd.DataFrame()
rr['Prolific_ID'] = finaldata['Prolific_ID']
rr['RR_score']= RR_score["RR_totalscore"]
rr.to_csv('%s/rr.csv' %(path.parent), index=False)

# selfreportdata = pd.read_csv('%s/selfreportdata_master.csv' %(path.parent))
# selfreportdata['RR'] = RR_score["RR_totalscore"]
# selfreportdata.to_csv('%s/selfreportdata_master.csv' %(path.parent), index=False)



In [19]:
#columns list
DAST_cols = [col for col in alldata.columns if 'DAST_' in col]
ProlificID_cols = [col for col in alldata.columns if 'Prolific_ID' in col]


DAST = alldata.filter(regex='DAST_|Prolific_ID')
#%%
DAST_clean = pd.DataFrame()

for i in range(0,len(DAST)):
    if DAST.loc[i,'Prolific_ID'] in participants['PROLIFIC_ID'].values:
       DAST_clean[i] = DAST.loc[i]    

finaldata = pd.DataFrame()


DAST_clean = DAST_clean.transpose()
DAST_clean = DAST_clean.reset_index()
finaldata['Prolific_ID'] = DAST_clean['Prolific_ID']
DAST_clean = DAST_clean.drop(['index'], axis = 1)
DAST_clean = DAST_clean.drop(['Prolific_ID'], axis = 1)
DAST_clean= DAST_clean.astype(int)
#%%
DAST_score = pd.DataFrame(columns = DAST_clean.columns, index = DAST_clean.index)

#%%
reverse_score= [4, 5, 7]

for k in range(0,len(DAST_clean)):
    for i in range (1,len(DAST_clean.columns)+1):
        if i in reverse_score:
            if DAST_clean.loc[k,'DAST_' + str(i)] == 0:
                DAST_score['DAST_' + str(i)][k] = 1
            elif DAST_clean.loc[k,'DAST_'+ str(i)] == 1:
                DAST_score['DAST_' + str(i)][k] = 0
            
        else:
            DAST_score['DAST_' +str(i)][k] = DAST_clean.loc[k,'DAST_'+ str(i)]
DAST_score= DAST_score.astype(int)
DAST_score["DAST"] = DAST_score.sum(axis=1)
#%%
dast = pd.DataFrame()
dast['Prolific_ID'] = finaldata['Prolific_ID']
dast['DAST_score']= DAST_score["DAST"]
dast.to_csv('%s/dast.csv' %(path.parent), index=False)

#selfreportdata = pd.read_csv('%s/selfreportdata_master_DF.csv' %(path.parent))
#selfreportdata['DAST'] = DAST_score["DAST"]
#selfreportdata.to_csv('%s/selfreportdata_master_DF.csv' %(path.parent), index=False)


In [22]:

#columns list
DII_cols = [col for col in alldata.columns if 'DII_' in col]
ProlificID_cols = [col for col in alldata.columns if 'Prolific_ID' in col]

DII = alldata.filter(regex='DII_|Prolific_ID')

#%%
DII_clean = pd.DataFrame()


for i in range(0, len(DII)):
    if DII.loc[i,'Prolific_ID'] in participants['PROLIFIC_ID'].values:
       DII_clean[i] = DII.loc[i]    

finaldata = pd.DataFrame()


DII_clean = DII_clean.transpose()
DII_clean = DII_clean.reset_index()
finaldata['Prolific_ID'] = DII_clean['Prolific_ID']
DII_clean = DII_clean.drop(['index'], axis = 1)
DII_clean = DII_clean.drop(['Prolific_ID'], axis = 1)
DII_clean = DII_clean.replace(np.nan, 0)
DII_clean = DII_clean.astype(int)
#%%
recode = [4, 7, 14]
for k in range(0, len(DII_clean)):
    for i in range (1,len(DII_clean.columns)+1):
        if i in recode:
            if DII_clean.loc[k, 'DII_' + str(i)] == 1:
                DII_clean['DII_' + str(i)][k] = 0
            elif DII_clean.loc[k, 'DII_' + str(i)] == 2:
                DII_clean['DII_' + str(i)][k] = 0
            elif DII_clean.loc[k, 'DII_' + str(i)] == 3:
                DII_clean['DII_' + str(i)][k] = 0
            elif DII_clean.loc[k, 'DII_' + str(i)] == 4:
                DII_clean['DII_' + str(i)][k] = 1
            elif DII_clean.loc[k, 'DII_' + str(i)] == 5:
                DII_clean['DII_' + str(i)][k] = 1
        else:
           DII_clean['DII_' +str(i)][k] = DII_clean.loc[k,'DII_'+ str(i)]
#%%
DII_score = pd.DataFrame(columns = DII_clean.columns, index = DII_clean.index)
#%%
functionalreverse = [2,3,8,11,12]
dysfunctionalreverse = [4,22,23]

for k in range(0, len(DII_clean)):
    for i in range (1,len(DII_clean.columns)+1):
        if i in functionalreverse:
            if DII_clean.loc[k,'DII_' + str(i)] == 0:
                DII_score['DII_' + str(i)][k] = 1
            elif DII_clean.loc[k,'DII_' + str(i)] == 1:
                 DII_score['DII_' + str(i)][k] = 0
        else:
            DII_score['DII_' +str(i)][k] = DII_clean.loc[k,'DII_'+ str(i)]
            
for k in range(0, len(DII_clean)):
    for i in range (1,len(DII_clean.columns)+1):
        if i in dysfunctionalreverse:
            if DII_clean.loc[k,'DII_' + str(i)] == 0:
                DII_score['DII_' + str(i)][k] = 1
            elif DII_clean.loc[k,'DII_' + str(i)] == 1:
                 DII_score['DII_' + str(i)][k] = 0
        else:
            DII_score['DII_' +str(i)][k] = DII_clean.loc[k,'DII_'+ str(i)]
        
DII_score = DII_score.astype(int)
# SRQ_negsocpot["SRQ_negsocpot"] = SRQ_negsocpot.sum(axis=1)/5
#%%
DII_functionalimpulsivity = pd.DataFrame()
DII_functionalimpulsivity['DII_2'] = DII_clean['DII_2']
DII_functionalimpulsivity['DII_3'] = DII_clean['DII_3']
DII_functionalimpulsivity['DII_8'] = DII_clean['DII_8']
DII_functionalimpulsivity['DII_11'] = DII_clean['DII_11']
DII_functionalimpulsivity['DII_12'] = DII_clean['DII_12']
DII_functionalimpulsivity['DII_5'] = DII_clean['DII_5']
DII_functionalimpulsivity['DII_19'] = DII_clean['DII_19']
DII_functionalimpulsivity['DII_15'] = DII_clean['DII_15']
DII_functionalimpulsivity['DII_6'] = DII_clean['DII_6']
DII_functionalimpulsivity['DII_16'] = DII_clean['DII_16']
DII_functionalimpulsivity['DII_20'] = DII_clean['DII_20']

DII_functionalimpulsivity = DII_functionalimpulsivity.astype(int)
DII_score["DII_functionalimpulsivity"] = DII_functionalimpulsivity.sum(axis=1)

#%%
DII_dysfuntionalimpulsivity = pd.DataFrame()
DII_dysfuntionalimpulsivity['DII_4'] = DII_clean['DII_4']
DII_dysfuntionalimpulsivity['DII_21'] = DII_clean['DII_21']
DII_dysfuntionalimpulsivity['DII_10'] = DII_clean['DII_10']
DII_dysfuntionalimpulsivity['DII_22'] = DII_clean['DII_22']
DII_dysfuntionalimpulsivity['DII_23'] = DII_clean['DII_23']
DII_dysfuntionalimpulsivity['DII_7'] = DII_clean['DII_7']
DII_dysfuntionalimpulsivity['DII_1'] = DII_clean['DII_1']
DII_dysfuntionalimpulsivity['DII_18'] = DII_clean['DII_18']
DII_dysfuntionalimpulsivity['DII_17'] = DII_clean['DII_17']
DII_dysfuntionalimpulsivity['DII_9'] = DII_clean['DII_9']
DII_dysfuntionalimpulsivity['DII_13'] = DII_clean['DII_13']

DII_dysfuntionalimpulsivity = DII_dysfuntionalimpulsivity.astype(int)
DII_score["DII_dysfuntionalimpulsivity"] = DII_dysfuntionalimpulsivity.sum(axis=1)

#%%

dii = pd.DataFrame()
dii['Prolific_ID'] = finaldata['Prolific_ID']
dii['DII_dysfuntionalimpulsivity']= DII_score["DII_dysfuntionalimpulsivity"]
dii['DII_functionalimpulsivity']= DII_score["DII_functionalimpulsivity"]
dii.to_csv('%s/DII.csv' %(path.parent), index=False)

#selfreportdata = pd.read_csv('%s/selfreportdata_master.csv' %(path.parent))
#selfreportdata['DII_functionalimpulsivity'] = DII_score["DII_functionalimpulsivity"]
#selfreportdata['DII_dysfunctionalimpulsivity'] = DII_score["DII_dysfuntionalimpulsivity"]
#selfreportdata.to_csv('%s/selfreportdata_master.csv' %(path.parent), index=False)


In [23]:
#columns list
LSAS_avoidancecols = [col for col in alldata.columns if 'LSAS_1_' in col]
LSAS_fearcols = [col for col in alldata.columns if 'LSAS_2_' in col]
ProlificID_cols = [col for col in alldata.columns if 'Prolific_ID' in col]


LSAS = alldata.filter(regex='LSAS_|Prolific_ID')


# %%
LSAS_clean = pd.DataFrame()


for i in range(0, len(LSAS)):
    if LSAS.loc[i,'Prolific_ID'] in participants['PROLIFIC_ID'].values:
       LSAS_clean[i] = LSAS.loc[i]

finaldata = pd.DataFrame()


LSAS_clean = LSAS_clean.transpose()
LSAS_clean = LSAS_clean.reset_index()
finaldata['Prolific_ID'] = LSAS_clean['Prolific_ID']
LSAS_clean = LSAS_clean.drop(['index'], axis = 1)
LSAS_clean = LSAS_clean.drop(['Prolific_ID'], axis = 1)
LSAS_clean = LSAS_clean.replace(np.nan, 0)
#%%

fear0 = LSAS_clean[LSAS_fearcols]
avoidance0 = LSAS_clean[LSAS_avoidancecols]

for k in range(0, len(fear0)):
    for i in range (1,len(fear0.columns)+1):
            if LSAS_clean.loc[k, 'LSAS_1_' + str(i)] == 1:
                LSAS_clean['LSAS_1_' + str(i)][k] = 0
            elif LSAS_clean.loc[k, 'LSAS_1_' + str(i)] == 2:
                LSAS_clean['LSAS_1_' + str(i)][k] = 1
            elif LSAS_clean.loc[k, 'LSAS_1_' + str(i)] == 3:
                LSAS_clean['LSAS_1_' + str(i)][k] = 2
            elif LSAS_clean.loc[k, 'LSAS_1_' + str(i)] == 4:
                LSAS_clean['LSAS_1_' + str(i)][k] = 3


for k in range(0, len(avoidance0)):
    for i in range (1,len(avoidance0.columns)+1):
            if LSAS_clean.loc[k, 'LSAS_2_' + str(i)] == 1:
                LSAS_clean['LSAS_2_' + str(i)][k] = 0
            elif LSAS_clean.loc[k, 'LSAS_2_' + str(i)] == 2:
                LSAS_clean['LSAS_2_' + str(i)][k] = 1
            elif LSAS_clean.loc[k, 'LSAS_2_' + str(i)] == 3:
                LSAS_clean['LSAS_2_' + str(i)][k] = 2
            elif LSAS_clean.loc[k, 'LSAS_2_' + str(i)] == 4:
                LSAS_clean['LSAS_2_' + str(i)][k] = 3
            #elif LSAS_clean.loc[k, 'DII_' + str(i)] == 5:
#%%
LSAS_score = pd.DataFrame(columns = LSAS_clean.columns, index = LSAS_clean.index)

#%%
LSAS_avoidance = pd.DataFrame()
LSAS_avoidance['LSAS_1_1'] = avoidance0['LSAS_1_1']
LSAS_avoidance['LSAS_1_2'] = avoidance0['LSAS_1_2']
LSAS_avoidance['LSAS_1_3'] = avoidance0['LSAS_1_3']
LSAS_avoidance['LSAS_1_4'] = avoidance0['LSAS_1_4']
LSAS_avoidance['LSAS_1_5'] = avoidance0['LSAS_1_5']
LSAS_avoidance['LSAS_1_6'] = avoidance0['LSAS_1_6']
LSAS_avoidance['LSAS_1_7'] = avoidance0['LSAS_1_7']
LSAS_avoidance['LSAS_1_8'] = avoidance0['LSAS_1_8']
LSAS_avoidance['LSAS_1_9'] = avoidance0['LSAS_1_9']
LSAS_avoidance['LSAS_1_10'] = avoidance0['LSAS_1_10']
LSAS_avoidance['LSAS_1_11'] = avoidance0['LSAS_1_11']
LSAS_avoidance['LSAS_1_12'] = avoidance0['LSAS_1_12']
LSAS_avoidance['LSAS_1_13'] = avoidance0['LSAS_1_13']
LSAS_avoidance['LSAS_1_14'] = avoidance0['LSAS_1_14']
LSAS_avoidance['LSAS_1_15'] = avoidance0['LSAS_1_15']
LSAS_avoidance['LSAS_1_16'] = avoidance0['LSAS_1_16']
LSAS_avoidance['LSAS_1_17'] = avoidance0['LSAS_1_17']
LSAS_avoidance['LSAS_1_18'] = avoidance0['LSAS_1_18']
LSAS_avoidance['LSAS_1_19'] = avoidance0['LSAS_1_19']
LSAS_avoidance['LSAS_1_20'] = avoidance0['LSAS_1_20']
LSAS_avoidance['LSAS_1_21'] = avoidance0['LSAS_1_21']
LSAS_avoidance['LSAS_1_22'] = avoidance0['LSAS_1_22']
LSAS_avoidance['LSAS_1_23'] = avoidance0['LSAS_1_23']
LSAS_avoidance['LSAS_1_24'] = avoidance0['LSAS_1_24']

LSAS_avoidance= LSAS_avoidance.astype(int)

#%%
LSAS_fear = pd.DataFrame()
LSAS_fear['LSAS_2_1'] = fear0['LSAS_2_1']
LSAS_fear['LSAS_2_2'] = fear0['LSAS_2_2']
LSAS_fear['LSAS_2_3'] = fear0['LSAS_2_3']
LSAS_fear['LSAS_2_4'] = fear0['LSAS_2_4']
LSAS_fear['LSAS_2_5'] = fear0['LSAS_2_5']
LSAS_fear['LSAS_2_6'] = fear0['LSAS_2_6']
LSAS_fear['LSAS_2_7'] = fear0['LSAS_2_7']
LSAS_fear['LSAS_2_8'] = fear0['LSAS_2_8']
LSAS_fear['LSAS_2_9'] = fear0['LSAS_2_9']
LSAS_fear['LSAS_2_10'] = fear0['LSAS_2_10']
LSAS_fear['LSAS_2_11'] = fear0['LSAS_2_11']
LSAS_fear['LSAS_2_12'] = fear0['LSAS_2_12']
LSAS_fear['LSAS_2_13'] = fear0['LSAS_2_13']
LSAS_fear['LSAS_2_14'] = fear0['LSAS_2_14']
LSAS_fear['LSAS_2_15'] = fear0['LSAS_2_15']
LSAS_fear['LSAS_2_16'] = fear0['LSAS_2_16']
LSAS_fear['LSAS_2_17'] = fear0['LSAS_2_17']
LSAS_fear['LSAS_2_18'] = fear0['LSAS_2_18']
LSAS_fear['LSAS_2_19'] = fear0['LSAS_2_19']
LSAS_fear['LSAS_2_20'] = fear0['LSAS_2_20']
LSAS_fear['LSAS_2_21'] = fear0['LSAS_2_21']
LSAS_fear['LSAS_2_22'] = fear0['LSAS_2_22']
LSAS_fear['LSAS_2_23'] = fear0['LSAS_2_23']
LSAS_fear['LSAS_2_24'] = fear0['LSAS_2_24']

LSAS_fear= LSAS_fear.astype(int)

#%%
LSAS_score =  pd.concat([LSAS_fear, LSAS_avoidance], axis=1)
LSAS_score["LSAS_avoidance"] = LSAS_avoidance.sum(axis=1)
LSAS_score["LSAS_fear"] = LSAS_fear.sum(axis=1)
LSAS_score["LSAS_total"] = LSAS_score['LSAS_avoidance'] + LSAS_score['LSAS_fear']

#%%
lsas = pd.DataFrame()
lsas['Prolific_ID'] = finaldata['Prolific_ID']
lsas['"LSAS_avoidance"']= LSAS_score["LSAS_avoidance"]
lsas['LSAS_fear']= LSAS_score["LSAS_fear"]
lsas['LSAS_total'] = LSAS_score['LSAS_total']
lsas.to_csv('%s/LSAS.csv' %(path.parent), index=False)
#selfreportdata = pd.read_csv('%s/selfreportdata_master.csv' %(path.parent))
#selfreportdata['LSAS_avoidance'] = LSAS_score["LSAS_avoidance"]
#selfreportdata['LSAS_fear'] = LSAS_score["LSAS_fear"]
#selfreportdata['LSAS_total'] = LSAS_score["LSAS_total"]
#selfreportdata.to_csv('%s/selfreportdata_master.csv' %(path.parent), index=False)


KeyError: 'LSAS_1_1'

In [24]:

#columns list
MSPSS_cols = [col for col in alldata.columns if 'MSPSS_' in col]
ProlificID_cols = [col for col in alldata.columns if 'Prolific_ID' in col]

MSPSS = alldata.filter(regex='MSPSS_|Prolific_ID')

#%%
MSPSS_clean = pd.DataFrame()


for i in range(0, len(MSPSS)):
    if MSPSS.loc[i,'Prolific_ID'] in participants['PROLIFIC_ID'].values:
       MSPSS_clean[i] = MSPSS.loc[i]    

finaldata = pd.DataFrame()


MSPSS_clean = MSPSS_clean.transpose()
MSPSS_clean = MSPSS_clean.reset_index()
finaldata['Prolific_ID'] = MSPSS_clean['Prolific_ID']
MSPSS_clean = MSPSS_clean.drop(['index'], axis = 1)
MSPSS_clean = MSPSS_clean.drop(['Prolific_ID'], axis = 1)
MSPSS_clean = MSPSS_clean.replace(np.nan, 0)
MSPSS_clean = MSPSS_clean.astype(int)

#%%
MSPSS_score = pd.DataFrame(columns = MSPSS_clean.columns, index = MSPSS_clean.index)

#%%

for k in range(0,len(MSPSS_clean)):
    for i in range (1,len(MSPSS_clean.columns)+1):

        MSPSS_score['MSPSS_' +str(i)][k] = MSPSS_clean.loc[k,'MSPSS_'+ str(i)]


MSPSS_score["MSPSS_totalscore"] = MSPSS_score.sum(axis=1)/12

#%%

mspss = pd.DataFrame()
mspss['Prolific_ID'] = finaldata['Prolific_ID']
mspss['MSPSS_score']= MSPSS_score["MSPSS_totalscore"]
mspss.to_csv('%s/mspss.csv' %(path.parent), index=False)

# selfreportdata = pd.read_csv('%s/selfreportdata_master.csv' %(path.parent))
# selfreportdata['MSPSS'] = MSPSS_score["MSPSS_totalscore"]
# selfreportdata.to_csv('%s/selfreportdata_master.csv' %(path.parent), index=False)
